In [1]:
from web3 import Web3
import os
from dotenv import load_dotenv
import pandas as pd
import prophet
import numpy as np
import datetime as dt
from datetime import timedelta
import prophet

from eth_account import Account
from web3.middleware import geth_poa_middleware
from eth_abi import decode
from eth_utils import decode_hex, to_text

import requests
import random
import json

from dune_client.client import DuneClient

import plotly.graph_objs as go

In [2]:
load_dotenv()

ETHERSCAN_KEY = os.getenv("ETHERSCAN_KEY")
COINGECKO_API_KEY = os.getenv("COINGECKO_API_KEY")
DUNE_API_KEY = os.getenv('DUNE_API_KEY')
FLIPSIDE_API_KEY=os.getenv('FLIPSIDE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

GAS_ACCOUNTANT = os.getenv("GAS_ACCOUNTANT")
GAS_RESERVE = os.getenv('GAS_RESERVE')
ACCOUNT_ADDRESS = os.getenv("ACCOUNT_ADDRESS")
PRIVATE_KEY = os.getenv("PRIVATE_KEY")
YIELD_FARM_ADDRESS = os.getenv("YIELD_FARM_ADDRESS")
STAKING_CONTRACT = os.getenv("STAKING_CONTRACT")
SEPOLIA_GATEWAY = os.getenv("SEPOLIA_GATEWAY")

In [3]:
os.chdir('..')

api = False

In [4]:
abi_path = r'gas_accountant_contracts\contracts\artifacts'
abi_paths = []  # Assuming GAS_ACCOUNTANT_ABI_PATH is predefined

for file in os.listdir(abi_path):
    if file.endswith('.json') and "metadata" not in file:  # Exclude metadata files
        abi_paths.append(os.path.join(abi_path, file))  # Add full path

print(abi_paths)  # Debug: Check the final list

abis = {}

for path in abi_paths:
    filename = os.path.basename(path)  # Extract filename (e.g., "YieldVault.json")
    name = os.path.splitext(filename)[0]  # Remove .json extension (e.g., "YieldVault")

    with open(path, "r") as file:
        abis[name] = json.load(file)  # Use name as key

print(abis)  # Debug output

['gas_accountant_contracts\\contracts\\artifacts\\GasReserve.json', 'gas_accountant_contracts\\contracts\\artifacts\\IGasReserve.json', 'gas_accountant_contracts\\contracts\\artifacts\\ILiquidStaking.json', 'gas_accountant_contracts\\contracts\\artifacts\\LiquidStaking.json', 'gas_accountant_contracts\\contracts\\artifacts\\StETHStrategy.json', 'gas_accountant_contracts\\contracts\\artifacts\\TestBTC.json', 'gas_accountant_contracts\\contracts\\artifacts\\TestETH.json', 'gas_accountant_contracts\\contracts\\artifacts\\YieldVault.json']
{'GasReserve': {'deploy': {'VM:-': {'linkReferences': {}, 'autoDeployLib': True}, 'main:1': {'linkReferences': {}, 'autoDeployLib': True}, 'ropsten:3': {'linkReferences': {}, 'autoDeployLib': True}, 'rinkeby:4': {'linkReferences': {}, 'autoDeployLib': True}, 'kovan:42': {'linkReferences': {}, 'autoDeployLib': True}, 'goerli:5': {'linkReferences': {}, 'autoDeployLib': True}, 'Custom': {'linkReferences': {}, 'autoDeployLib': True}}, 'data': {'bytecode': {'

In [5]:
abis.keys()

dict_keys(['GasReserve', 'IGasReserve', 'ILiquidStaking', 'LiquidStaking', 'StETHStrategy', 'TestBTC', 'TestETH', 'YieldVault'])

In [6]:
w3 = Web3(Web3.HTTPProvider(SEPOLIA_GATEWAY))

ACCOUNT = Account.from_key(PRIVATE_KEY)

w3.eth.default_account = ACCOUNT.address
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [7]:
def dune_api_results(query_num, save_csv=False, csv_path=None):
    results = dune.get_latest_result(query_num)
    df = pd.DataFrame(results.result.rows)

    if save_csv and csv_path:
        df.to_csv(csv_path, index=False)
    return df

In [8]:
def get_token_price(token='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'):
    url = f"https://api.coingecko.com/api/v3/simple/token_price/ethereum?contract_addresses={token}&vs_currencies=usd"

    headers = {
        "accept": "application/json",
        "x-cg-demo-api-key": COINGECKO_API_KEY
    }

    response = requests.get(url, headers=headers)

    

    eth_data = response.json()

    eth_df = pd.DataFrame(eth_data)
    eth_usd = eth_df[f'{token}'].values[0]

    print(eth_usd)

    return eth_usd

In [9]:
def process_transaction(tx):
    gas_price = float(tx.get("gasPrice", np.nan))  # Allow gasPrice to be NaN
    gas_used = int(tx["gasUsed"])
    
    return {
        "blockNumber": int(tx["blockNumber"]),
        "timestamp": dt.datetime.utcfromtimestamp(int(tx["timeStamp"])),
        "transaction_hash": tx["hash"],
        "from": tx["from"],
        "to": tx["to"] if tx.get("to") else "Contract Deployment",
        "gas": int(tx["gas"]),
        "gasPrice": gas_price,  # Leave NaN if missing
        "gasUsed": gas_used,
        "tx_fee": (gas_used * gas_price / 1e18) if not np.isnan(gas_price) else np.nan,  # Handle NaN in calculation
        "contractAddress": tx.get("contractAddress"),  # Default to None if contractAddress is missing
    }


In [10]:
def get_tx_and_log_with_pagination(contract_address, start_block, end_block, etherscan_api_key,module='account',action='txlist'):
    """
    Fetch logs for a contract address with pagination support from the Etherscan API.

    Parameters:
        contract_address (str): The contract address to fetch logs for.
        start_block (int): The starting block number.
        end_block (int): The ending block number or "latest".
        etherscan_api_key (str): Your Etherscan API key.
        topic_filters (dict, optional): A dictionary of topic filters, e.g.,
            {
                "topic0": "0xe085b50dde9f45e2f6290b8f6eadc05e9f66d77b30d750cb3930c5e3430b9c1e",
                "topic1": "0x0000000000000000000000002102240d1a36a9dc9f3a4d07ee9251cb723aca89",
                "topic2": None,
                "topic3": None,
                "topic0_1_opr": "and"
            }

    Returns:
        list: A list of logs fetched from the API.
    """
    if module == 'account':
        data_pulled = 'tx'
    else:
        data_pulled = 'log'

    base_url = "https://api-sepolia.etherscan.io/api"
    logs = []  # To store all logs
    page = 1
    offset = 1000  # Max records per page

    while True:
        # Construct the base URL
        url = (
            f"{base_url}?module={module}"
            f"&action={action}"
            f"&address={contract_address}"
            f"&fromBlock={start_block}"
            f"&toBlock={end_block}"
            f"&page={page}"
            f"&offset={offset}"
            f"&apikey={etherscan_api_key}"
        )

        try:
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()

            data = response.json()
            if data["status"] != "1":  # Etherscan returns "1" for success
                print(f"No more {data_pulled}s or error: {data.get('message', 'Unknown error')}")
                break

            logs.extend(data["result"])  # Append the logs to the list
            print(f"Fetched {len(data['result'])} {data_pulled}s from page {page}.")

            # Stop if fewer than `offset` logs are returned
            if len(data["result"]) < offset:
                print(f"All {data_pulled}s fetched.")
                break

            page += 1  # Move to the next page

        except requests.RequestException as e:
            print(f"Error while fetching {data_pulled}s: {e}")
            break

    return logs

In [11]:
def parse_gas_log(log):
    """
    Parse a gas request log entry.

    Parameters:
        log (dict): A raw log entry from Etherscan.

    Returns:
        dict: Parsed log data.
    """
    # Extract transaction hash
    tx_hash = log["transactionHash"]

    # Extract requester's address from topic[1]
    requester = "0x" + log["topics"][1][-40:]

    # Decode data fields
    data = log["data"][2:]  # Remove "0x"
    timestamp = int(data[0:64], 16)  # Convert hex to int
    amount = int(data[64:128], 16) / 1e18  # Convert to ETH

    # Convert timestamp to human-readable format
    timestamp_human = dt.datetime.utcfromtimestamp(timestamp)

    return {
        "timestamp": timestamp_human,
        "transaction_hash": tx_hash,
        "requester": requester,
        "gas_amount_eth": amount,
    }


In [12]:
def token_prices(token_addresses, network, start_date):
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    """
    Generate a SQL query to get historical price data for given token addresses from a specific start date.

    Parameters:
    - token_addresses (list): List of token addresses.
    - start_date (str): Start date in 'YYYY-MM-DD' format.

    Returns:
    - str: The SQL query string.
    """
    # Format the addresses into the SQL VALUES clause
    addresses_clause = ", ".join(f"(LOWER('{address}'))" for address in token_addresses)

    beginning = f"'{start_date.strftime('%Y-%m-%d %H:%M:%S')}'"
    print('Beginning:', beginning)
    
    prices_query = f"""
    WITH addresses AS (
        SELECT column1 AS token_address 
        FROM (VALUES
            {addresses_clause}
        ) AS tokens(column1)
    )

    SELECT 
        hour,
        symbol,
        price
    FROM 
        {network}.price.ez_prices_hourly
    WHERE 
        token_address IN (SELECT token_address FROM addresses)
        AND hour >= DATE_TRUNC('hour', TO_TIMESTAMP({beginning}, 'YYYY-MM-DD HH24:MI:SS'))
    ORDER BY 
        hour DESC, symbol
    """

    return prices_query

In [13]:
def flipside_api_results(query, api_key, attempts=10, delay=30):
    """
    Creates and retrieves results for a query using Flipside's JSON-RPC API with pagination.

    Parameters:
    - query: str, the SQL query to execute.
    - api_key: str, your Flipside API key.
    - attempts: int, number of attempts to poll for query completion.
    - delay: int, delay (in seconds) between polling attempts.

    Returns:
    - pd.DataFrame: DataFrame containing the query results.
    """
    import requests
    import time
    import pandas as pd

    # Step 1: Create the query
    url = "https://api-v2.flipsidecrypto.xyz/json-rpc"
    headers = {
        "Content-Type": "application/json",
        "x-api-key": api_key
    }
    payload = {
        "jsonrpc": "2.0",
        "method": "createQueryRun",
        "params": [
            {
                "resultTTLHours": 1,
                "maxAgeMinutes": 0,
                "sql": query,
                "tags": {"source": "python-script", "env": "production"},
                "dataSource": "snowflake-default",
                "dataProvider": "flipside"
            }
        ],
        "id": 1
    }

    response = requests.post(url, headers=headers, json=payload)
    response_data = response.json()

    if 'error' in response_data:
        raise Exception(f"Error creating query: {response_data['error']['message']}")

    query_run_id = response_data.get('result', {}).get('queryRun', {}).get('id')
    if not query_run_id:
        raise KeyError(f"Query creation failed. Response: {response_data}")

    # Step 2: Poll for query completion
    for attempt in range(attempts):
        status_payload = {
            "jsonrpc": "2.0",
            "method": "getQueryRunResults",
            "params": [
                {
                    "queryRunId": query_run_id,
                    "format": "json",
                    "page": {"number": 1, "size": 10000}
                }
            ],
            "id": 1
        }
        response = requests.post(url, headers=headers, json=status_payload)
        resp_json = response.json()

        if 'result' in resp_json and 'rows' in resp_json['result']:
            # Step 3: Handle pagination
            all_rows = []
            page_number = 1

            while True:
                status_payload["params"][0]["page"]["number"] = page_number
                response = requests.post(url, headers=headers, json=status_payload)
                resp_json = response.json()

                if 'result' in resp_json and 'rows' in resp_json['result']:
                    rows = resp_json['result']['rows']
                    if not rows:
                        break  # No more rows to fetch
                    all_rows.extend(rows)
                    page_number += 1
                else:
                    break

            # Convert the rows to a DataFrame
            return pd.DataFrame(all_rows)

        if 'error' in resp_json and 'not yet completed' in resp_json['error'].get('message', '').lower():
            time.sleep(delay)  # Wait before retrying
        else:
            raise Exception(f"Unexpected error while fetching query results: {resp_json}")

    raise TimeoutError(f"Query did not complete after {attempts} attempts.")

In [14]:
def to_time(df):
    time_cols = ['date','dt','hour','time','day','month','year','week','timestamp','date(utc)','block_timestamp']
    for col in df.columns:
        if col.lower() in time_cols and col.lower() != 'timestamp':
            df[col] = pd.to_datetime(df[col])
            df.set_index(col, inplace=True)
        elif col.lower() == 'timestamp':
            df[col] = pd.to_datetime(df[col], unit='ms')
            df.set_index(col, inplace=True)
    print(df.index)
    return df 

def clean_prices(prices_df):
    print('cleaning prices')
    # Pivot the dataframe
    breakpoint()
    prices_df = prices_df.drop_duplicates(subset=['hour', 'symbol'])
    prices_df_pivot = prices_df.pivot(
        index='hour',
        columns='symbol',
        values='price'
    )
    prices_df_pivot = prices_df_pivot.reset_index()

    # Rename the columns by combining 'symbol' with a suffix
    prices_df_pivot.columns = ['dt'] + [f'{col}_price' for col in prices_df_pivot.columns[1:]]
    
    print(f'cleaned prices: {prices_df_pivot}')
    return prices_df_pivot

In [15]:
def data_processing(df,dropna=True):
    df.columns=df.columns.str.lower()
    clean_df = clean_prices(df)
    clean_df = to_time(clean_df)
    if dropna == True:
        clean_df = clean_df.dropna(axis=1, how='any')

    if '__row_index' in clean_df.columns:
        clean_df.drop(columns=['__row_index'], inplace=True)

    return clean_df

# Gas Price Forecasting

In [16]:
def get_sepolia_gas_history(api, term='long_term'):
    lt_sepolia_gas_data_path = 'data/sepolia_gas_metrics.csv'
    st_sepolia_gas_data_path = 'data/current_sepolia_gas_metrics.csv'

    if api:
        if term == 'long_term':
            query = 4622627  # 2 Years worth
            sepolia_gas_data = dune_api_results(query, True, lt_sepolia_gas_data_path)
            sepolia_gas_data.to_csv(lt_sepolia_gas_data_path, index=False)  # ✅ Moved inside block
        
        elif term == 'short_term':
            query = 4650506  # Runs daily at midnight for latest prices
            sepolia_gas_data = dune_api_results(query, True, st_sepolia_gas_data_path)
            sepolia_gas_data.to_csv(st_sepolia_gas_data_path, index=False)

        else:
            print('Pass "long_term" or "short_term" as parameters')
            return None

    else:
        if term == 'long_term':
            sepolia_gas_data = pd.read_csv(lt_sepolia_gas_data_path)
        elif term == 'short_term':
            sepolia_gas_data = pd.read_csv(st_sepolia_gas_data_path)
        else:
            print('Pass "long_term" or "short_term" as parameters')
            return None

    # Convert 'hour' column to datetime format
    sepolia_gas_data['hour'] = pd.to_datetime(sepolia_gas_data['hour'])

    return sepolia_gas_data

In [17]:
long_term_sepolia_gas = get_sepolia_gas_history(False,'long_term')

In [18]:
short_term_sepolia_gas = get_sepolia_gas_history(False,'short_term')

In [19]:
short_term_sepolia_gas

,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,2025-01-31 23:00:00+00:00,100000000000,1.975151e+09,901905452,1.224018e+09,2.485303e+09
1,2025-01-31 22:00:00+00:00,100000000000,2.000216e+09,815791549,1.259035e+09,2.527053e+09
2,2025-01-31 21:00:00+00:00,210000000000,2.858135e+09,1193908859,2.211837e+09,3.439532e+09
3,2025-01-31 20:00:00+00:00,210000000000,3.130228e+09,1858195062,2.491308e+09,3.632018e+09
4,2025-01-31 19:00:00+00:00,500000000000,7.116758e+09,1978633021,4.255715e+09,9.735677e+09
5,2025-01-31 18:00:00+00:00,614806191671,2.222849e+10,9790439166,1.720389e+10,2.915616e+10
6,2025-01-31 17:00:00+00:00,896177980852,1.029691e+10,6083004053,9.014917e+09,1.120849e+10
7,2025-01-31 16:00:00+00:00,1470000000000,1.616902e+10,9397889672,1.299390e+10,2.024024e+10
8,2025-01-31 15:00:00+00:00,1470000000000,8.295084e+09,4879524531,7.261535e+09,9.194055e+09
9,2025-01-31 14:00:00+00:00,892900071902,7.794638e+09,3365698853,6.126430e+09,8.781174e+09


In [20]:
long_term_sepolia_gas

,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,2025-01-30 04:00:00+00:00,100000000000,2.178866e+09,871441221,1.282759e+09,2.536024e+09
1,2025-01-30 03:00:00+00:00,222995523835,2.158460e+09,842989635,1.322712e+09,2.572218e+09
2,2025-01-30 02:00:00+00:00,300000000000,2.281369e+09,827189632,1.325706e+09,2.635527e+09
3,2025-01-30 01:00:00+00:00,200000000000,2.147583e+09,827189632,1.176685e+09,2.532289e+09
4,2025-01-30 00:00:00+00:00,290000000000,2.136518e+09,819280840,1.382348e+09,2.524868e+09
...,...,...,...,...,...,...
17280,2023-02-10 04:00:00+00:00,25000000000000,1.518708e+09,7,1.500000e+09,4.500000e+09
17281,2023-02-10 03:00:00+00:00,25000000000000,5.447670e+09,7,4.946813e+09,1.093770e+10
17282,2023-02-10 02:00:00+00:00,100000000000,4.350000e+09,7,2.776746e+09,5.178040e+09
17283,2023-02-10 01:00:00+00:00,100000000000,2.383351e+09,7,1.548750e+09,4.590795e+09


In [21]:
def set_global_seed(env, seed=20):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)


# Tracking the Test Protocol Gas Reverse

In [22]:
current_block = w3.eth.block_number
print(f"Current block number: {current_block}")

Current block number: 7613720


In [23]:
GAS_RESERVE

'0x0064c627A55AEF07f4aB6Fea62E516A76E38aC8A'

## We can track internal TX of gas reserve contract to see how much it spends

In [24]:
# tx_history = get_tx_with_pagination(GAS_RESERVE, current_block - 10000, current_block - 1, ETHERSCAN_KEY)
internal_tx_history = get_tx_and_log_with_pagination(GAS_RESERVE, current_block - 10000, current_block, ETHERSCAN_KEY, action='txlistinternal')


Fetched 1 txs from page 1.
All txs fetched.


In [25]:
processed_transactions = [process_transaction(tx) for tx in internal_tx_history]

# Convert to DataFrame
df = pd.DataFrame(processed_transactions)

dataset = df.copy()
dataset = dataset[dataset['from']==GAS_RESERVE.lower()]
dataset['hour'] = dataset['timestamp'].dt.strftime('%Y-%m-%d %H:00:00')

### Here we combine gas history to get full tx history

In [29]:
earliest_date = dataset['hour'].min()

In [30]:
eth_query = token_prices(['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'],'ethereum',earliest_date)
eth_df = flipside_api_results(api_key=FLIPSIDE_API_KEY,query=eth_query)
eth_df['hour'] = pd.to_datetime(eth_df['hour']).dt.strftime('%Y-%m-%d %H:00:00')
eth_df.set_index('hour',inplace=True)
eth_df_wide = data_processing(eth_df.reset_index())
eth_df_wide.index = eth_df_wide.index.strftime('%Y-%m-%d %H:00:00')

Beginning: '2025-01-31 21:00:00'
cleaning prices
cleaned prices:                     dt  WETH_price
0  2025-01-31 21:00:00     3322.87
1  2025-01-31 22:00:00     3327.19
2  2025-01-31 23:00:00     3309.11
3  2025-02-01 00:00:00     3296.75
DatetimeIndex(['2025-01-31 21:00:00', '2025-01-31 22:00:00',
               '2025-01-31 23:00:00', '2025-02-01 00:00:00'],
              dtype='datetime64[ns]', name='dt', freq=None)


In [31]:
short_term_sepolia_gas

,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,2025-01-31 23:00:00+00:00,100000000000,1.975151e+09,901905452,1.224018e+09,2.485303e+09
1,2025-01-31 22:00:00+00:00,100000000000,2.000216e+09,815791549,1.259035e+09,2.527053e+09
2,2025-01-31 21:00:00+00:00,210000000000,2.858135e+09,1193908859,2.211837e+09,3.439532e+09
3,2025-01-31 20:00:00+00:00,210000000000,3.130228e+09,1858195062,2.491308e+09,3.632018e+09
4,2025-01-31 19:00:00+00:00,500000000000,7.116758e+09,1978633021,4.255715e+09,9.735677e+09
5,2025-01-31 18:00:00+00:00,614806191671,2.222849e+10,9790439166,1.720389e+10,2.915616e+10
6,2025-01-31 17:00:00+00:00,896177980852,1.029691e+10,6083004053,9.014917e+09,1.120849e+10
7,2025-01-31 16:00:00+00:00,1470000000000,1.616902e+10,9397889672,1.299390e+10,2.024024e+10
8,2025-01-31 15:00:00+00:00,1470000000000,8.295084e+09,4879524531,7.261535e+09,9.194055e+09
9,2025-01-31 14:00:00+00:00,892900071902,7.794638e+09,3365698853,6.126430e+09,8.781174e+09


In [32]:
dataset

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour
0,7612904,2025-01-31 21:28:24,0xaf8ab04470639668568878222952df0a4a44bc10cb6e...,0x0064c627a55aef07f4ab6fea62e516a76e38ac8a,0x0448d01970801692c22ff2eccc570f2f7cd9b889,2300,NaN,55,NaN,,2025-01-31 21:00:00


In [33]:
short_term_sepolia_gas['hour'] = pd.to_datetime(short_term_sepolia_gas['hour']).dt.strftime('%Y-%m-%d %H:00:00')
dataset_merged = dataset.merge(
    short_term_sepolia_gas,
    on='hour',
    how='left'
)
print(dataset)

   blockNumber           timestamp  \
0      7612904 2025-01-31 21:28:24   

                                    transaction_hash  \
0  0xaf8ab04470639668568878222952df0a4a44bc10cb6e...   

                                         from  \
0  0x0064c627a55aef07f4ab6fea62e516a76e38ac8a   

                                           to   gas  gasPrice  gasUsed  \
0  0x0448d01970801692c22ff2eccc570f2f7cd9b889  2300       NaN       55   

   tx_fee contractAddress                 hour  
0     NaN                  2025-01-31 21:00:00  


In [34]:
dataset_merged['hour']

0    2025-01-31 21:00:00
Name: hour, dtype: object

In [35]:
dataset_merged['gasPrice'] = dataset_merged['gasPrice'].fillna(dataset_merged['median_gas_price'])
dataset_merged['tx_fee'] = dataset_merged['tx_fee'].fillna((dataset_merged['gasPrice'] * dataset_merged['gasUsed']) / 1e18)

In [36]:
eth_df_wide.reset_index(inplace=True)
eth_df_wide.rename(columns={'dt':'hour'},inplace=True)

In [37]:
dataset_merged

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price
0,7612904,2025-01-31 21:28:24,0xaf8ab04470639668568878222952df0a4a44bc10cb6e...,0x0064c627a55aef07f4ab6fea62e516a76e38ac8a,0x0448d01970801692c22ff2eccc570f2f7cd9b889,2300,2.858135e+09,55,1.571974e-07,,2025-01-31 21:00:00,210000000000,2.858135e+09,1193908859,2.211837e+09,3.439532e+09


In [38]:
eth_df_wide.columns

Index(['hour', 'WETH_price'], dtype='object')

In [39]:
dataset

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour
0,7612904,2025-01-31 21:28:24,0xaf8ab04470639668568878222952df0a4a44bc10cb6e...,0x0064c627a55aef07f4ab6fea62e516a76e38ac8a,0x0448d01970801692c22ff2eccc570f2f7cd9b889,2300,NaN,55,NaN,,2025-01-31 21:00:00


In [ ]:
dataset_merged=dataset_merged.merge(
    eth_df_wide,
    on='hour',
    how='left'
)

In [41]:
dataset_merged['tx_fee_usd'] = dataset_merged['tx_fee'] * dataset_merged['WETH_price']
dataset_merged['tx_fee_usd']

0    0.000522
Name: tx_fee_usd, dtype: float64

In [42]:
dataset_merged

,blockNumber,timestamp,transaction_hash,from,to,gas,gasPrice,gasUsed,tx_fee,contractAddress,hour,max_gas_price,median_gas_price,min_gas_price,p25_gas_price,p75_gas_price,WETH_price,tx_fee_usd
0,7612904,2025-01-31 21:28:24,0xaf8ab04470639668568878222952df0a4a44bc10cb6e...,0x0064c627a55aef07f4ab6fea62e516a76e38ac8a,0x0448d01970801692c22ff2eccc570f2f7cd9b889,2300,2.858135e+09,55,1.571974e-07,,2025-01-31 21:00:00,210000000000,2.858135e+09,1193908859,2.211837e+09,3.439532e+09,3322.87,0.000522


## We can also read the event logs to see gas requested, requester, and time requested

In [43]:
logs = get_tx_and_log_with_pagination(GAS_RESERVE, current_block - 10000, current_block, ETHERSCAN_KEY,module='logs', action='getLogs')

Fetched 1 logs from page 1.
All logs fetched.


In [44]:
parsed_logs = [parse_gas_log(log) for log in logs]
for log in parsed_logs:
    print(log)

{'timestamp': datetime.datetime(2025, 1, 31, 21, 28, 24), 'transaction_hash': '0xaf8ab04470639668568878222952df0a4a44bc10cb6e98d911a2c45bba85faea', 'requester': '0x0448d01970801692c22ff2eccc570f2f7cd9b889', 'gas_amount_eth': 0.01}


In [45]:
df = pd.DataFrame(parsed_logs)

# Convert timestamp to datetime format
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Set timestamp as the index
df.set_index("timestamp", inplace=True)

# Sort by timestamp
df = df.sort_index()

# Display the first few rows
print(df.head())

                                                      transaction_hash  \
timestamp                                                                
2025-01-31 21:28:24  0xaf8ab04470639668568878222952df0a4a44bc10cb6e...   

                                                      requester  \
timestamp                                                         
2025-01-31 21:28:24  0x0448d01970801692c22ff2eccc570f2f7cd9b889   

                     gas_amount_eth  
timestamp                            
2025-01-31 21:28:24            0.01  


## Here we combine gas history to get full tx history

# Tracking AI Agent Gas Costs

In [46]:
os.chdir('notebooks')